TripAdvisor-Top 30 Attractions

In [ ]:
# Connect to Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Import libraries
import requests
import pandas as pd
import csv
from bs4 import BeautifulSoup
from numpy import NAN

In [ ]:
# Define a user agent to qualify request
user_agent = ({"User-Agent":
                    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) \
                    AppleWebKit/537.36 (KHTML, like Gecko) \
                    Chrome/90.0.4430.212 Safari/537.36",
               "Accept-Language": "en-US, en;q=0.5"})

In [ ]:
# Import the list of main page links
df = pd.read_csv("/content/drive/MyDrive/Priceline/JintongYu/links.csv")
main_pages = df["TA_main_page"].tolist()

In [ ]:
# Define a function to extract the characters of the attractions on the 1st page for each city
def todo_30(main_page):
  try:
    # Send an HTTP GET request to the main link and retrieve the HTML content
    response = requests.get(main_page, headers=user_agent)
    html_content = response.content

    # Parse the HTML content to extract the subdirectory link
    soup = BeautifulSoup(html_content, "html.parser")
    subdirectory_link = soup.find("a", href=lambda href: href and "/Attractions-" in href).get("href")

    # Construct the full subdirectory link
    full_subdirectory_link = "https://www.tripadvisor.com" + subdirectory_link

    # Send an HTTP GET request to the subdirectory link and retrieve the HTML content
    response = requests.get(full_subdirectory_link, headers=user_agent)
    html_content = response.content

    # Send an HTTP GET request to the sub-subdirectory link and retrieve the HTML content
    soup = BeautifulSoup(html_content, "html.parser")
    sub_subdirectory_link = soup.find("a", href=lambda href: href and "/Attractions-" and "oa0" in href).get("href")
    full_sub_subdirectory_link = "https://www.tripadvisor.com" + sub_subdirectory_link
    response = requests.get(full_sub_subdirectory_link, headers=user_agent)
    html_content = response.content

    # Parse the HTML content of the sub-subdirectory link
    soup = BeautifulSoup(html_content, "html.parser")
    # Extract characters of the 1st page attractions (30) as a whole
    characters = []
    for tag in soup.find_all('div', {'class': 'alPVI eNNhq PgLKC tnGGX yzLvM'}):
      if tag.find('div', {'class': 'biGQs _P pZUbB hmDzD'}, recursive=False) is not None:
        character_tags = tag.find('div', {'class': 'biGQs _P pZUbB hmDzD'}, recursive=False)
    
        for tag in character_tags:
          content = tag.get_text()
          characters.append(content)

    return{
        "attractions_url": full_sub_subdirectory_link,
        "30_attractions_char": characters
    }
  except:
    return{
        "attractions_url": NAN,
        "30_attractions_char": NAN
        }

In [ ]:
# Extract "todo_30" for all cities
results = []
for main_page in main_pages:
  results.append(todo_30(main_page))

In [ ]:
# Turn dictionary into a dataframe
todo_30_df = pd.DataFrame.from_dict(results)
todo_30_df.head()

,attractions_url,30_attractions_char
0,https://www.tripadvisor.com/Attractions-g18779...,"[Historic Sites • Ancient Ruins, Ancient Ruins..."
1,https://www.tripadvisor.com/Attractions-g18749...,[Points of Interest & Landmarks • Architectura...
2,https://www.tripadvisor.com/Attractions-g29397...,"[Historic Sites • Architectural Buildings, His..."
3,https://www.tripadvisor.com/Attractions-g18784...,"[Religious Sites • Churches & Cathedrals, Poin..."
4,https://www.tripadvisor.com/Attractions-g18789...,"[Points of Interest & Landmarks, Art Museums, ..."


In [ ]:
# Check whether there are cities with empty target content
todo_30_df.describe(include='all')

,attractions_url,30_attractions_char
count,178,178
unique,178,178
top,https://www.tripadvisor.com/Attractions-g18779...,"[Historic Sites • Ancient Ruins, Ancient Ruins..."
freq,1,1


In [ ]:
# Concate the "todo_30" dataframe with the original city list
todo_30_df_concat = pd.concat([df.iloc[:, 0:5], todo_30_df], axis=1)
todo_30_df_concat.head()

,region_name,country_code,province_name,city_name,region_city_rank,attractions_url,30_attractions_char
0,SOUTHERN EUROPE,IT,LAZIO,ROME,1,https://www.tripadvisor.com/Attractions-g18779...,"[Historic Sites • Ancient Ruins, Ancient Ruins..."
1,SOUTHERN EUROPE,SP,CATALONIA,BARCELONA,2,https://www.tripadvisor.com/Attractions-g18749...,[Points of Interest & Landmarks • Architectura...
2,SOUTHERN EUROPE,TU,ISTANBUL,ISTANBUL,3,https://www.tripadvisor.com/Attractions-g29397...,"[Historic Sites • Architectural Buildings, His..."
3,SOUTHERN EUROPE,IT,LOMBARDIA,MILAN,4,https://www.tripadvisor.com/Attractions-g18784...,"[Religious Sites • Churches & Cathedrals, Poin..."
4,SOUTHERN EUROPE,IT,TOSCANA (TUSCANY),FLORENCE,5,https://www.tripadvisor.com/Attractions-g18789...,"[Points of Interest & Landmarks, Art Museums, ..."


In [ ]:
# Fill in missing values
todo_30_df_concat[todo_30_df_concat['30_attractions_char'].isnull()]

,Unnamed: 0,region_name,country_code,province_name,city_name,region_city_rank,attractions_url,30_attractions_char
52,52,UNITED STATES,US,FLORIDA,LAKE BUENA VISTA,13,NaN,NaN
101,101,UNITED STATES,US,CONNECTICUT,UNCASVILLE,62,NaN,NaN


In [ ]:
# Insert the corresponding links into the original data frame
todo_30_df_concat.at[52, 'attractions_url'] = "https://www.tripadvisor.com/Attractions-g23869903-Activities-Lake_Buena_Vista_Florida.html"
todo_30_df_concat.at[101, 'attractions_url'] = "https://www.tripadvisor.com/Attractions-g33952-Activities-Uncasville_Mystic_Country_Connecticut.html"

In [ ]:
# Create a list of links for missing values
missing_links = [
    "https://www.tripadvisor.com/Attractions-g23869903-Activities-Lake_Buena_Vista_Florida.html",
    "https://www.tripadvisor.com/Attractions-g33952-Activities-Uncasville_Mystic_Country_Connecticut.html"
]

# Define a function for scraping and parsing
def direct_scrape(missing_link):
  try:
    # Send an HTTP Get request to the link
    response = requests.get(missing_link, headers=user_agent)
    html_content = response.content

    # Parse the HTML content to extract the characters
    soup = BeautifulSoup(html_content, "html.parser")
    characters_m = []
    for tag in soup.find_all('div', {'class': 'alPVI eNNhq PgLKC tnGGX yzLvM'}):
      if tag.find('div', {'class': 'biGQs _P pZUbB hmDzD'}, recursive=False) is not None:
        character_tags = tag.find('div', {'class': 'biGQs _P pZUbB hmDzD'}, recursive=False)
    
        for tag in character_tags:
          content = tag.get_text()
          characters_m.append(content)
    return{
        "characters": characters_m
    }
  except:
    return{
        "characters": NAN
    }

# Scrape and parse
results_m = []
for missing_link in missing_links:
  results_m.append(direct_scrape(missing_link))

In [ ]:
# Check scraping results
results_m

[{'characters': ['Amusement & Theme Parks',
   'Dance Clubs & Discos',
   'Taxis & Shuttles',
   'Taxis & Shuttles',
   'Speciality & Gift Shops',
   'Private Tours',
   'Day Trips']},
 {'characters': ['Arenas & Stadiums',
   'Casinos',
   'History Museums',
   'Farms',
   'Parks',
   'Churches & Cathedrals',
   'Comedy Clubs',
   'Bars & Clubs • Comedy Clubs',
   'Game & Entertainment Centers',
   'Spas',
   'Bars & Clubs',
   'Spas',
   'Bars & Clubs',
   'Game & Entertainment Centers',
   'Taxis & Shuttles • Hiking & Camping Tours']}]

In [ ]:
# Convert the dictionary to a data frame
missing_df = pd.DataFrame.from_dict(results_m)
missing_df.head()

,characters
0,"[Amusement & Theme Parks, Dance Clubs & Discos..."
1,"[Arenas & Stadiums, Casinos, History Museums, ..."


In [ ]:
# Insert the characters into the original data frame
todo_30_df_concat.at[52, '30_attractions_char'] = missing_df['characters'][0]
todo_30_df_concat.at[101, '30_attractions_char'] = missing_df['characters'][1]

In [ ]:
# Save to a CSV file
todo_30_df_concat.to_csv("/content/drive/MyDrive/Priceline/JintongYu/todo_30.csv")